# Cleaning up currency data with pandas

# introduction

The other day,i was using pandas to clean some messy excel data that includes several thousand rows which has inconsistensly currency values.When i tried to clean it up,I realized that it was little more complicated than i thought.Coincidentally,couple of days later i found a twitter thread which shed some light on the issue iwas experiencing.This article summarises and decribes how to deal with currency fields and how to convert them into numeric value for further analysis.The concepts illustrated here can also apply to other types of pandas data cleanup tasks.

# About the data 

in this example,the data is mixture of currency labeled and non-currency labeled values.for a small example like we can sort it out in the source file.However,when you we have a large number of rows,we will have no choice but start with messy data and clean using pandas

In [10]:
#importing pandas library and reading the file
import pandas as pd
df_orig = pd.read_excel('sales_cleanup.xlsx')
df = df_orig.copy()

In [11]:
#viewing the first five rows of the dataset.
df.head()

,Customer,Sales
0,Jones Brothers,500
1,Beta Corp,"$1,000.00"
2,Globex Corp,300.1
3,Acme,$750.01
4,Initech,300


In [12]:
df.shape

(6, 2)

In [13]:
df

,Customer,Sales
0,Jones Brothers,500
1,Beta Corp,"$1,000.00"
2,Globex Corp,300.1
3,Acme,$750.01
4,Initech,300
5,Hooli,250


# one of the first thing i see after loading the data is the data types of the data

In [14]:
df.dtypes

Customer    object
Sales       object
dtype: object

Not suprisingly,the sales column is object.The '$' and ',' are the dead giveaways that the sales column is not numeric.More than likely we want to do some math on the column so lets try to convert it to float.

In [15]:
df['Sales'].astype('float')

ValueError: could not convert string to float: '$1,000.00'

The traceback includes a ValueError and shows that it could not convert the $1,000.00 string to a float. Ok. That should be easy to clean up.

# lets try removing ' $' and ',' using str.replace:

In [16]:
df['Sales'] = df['Sales'].str.replace(',','')
df['Sales'] = df['Sales'].str.replace('$','')

In [17]:
df['Sales']

0        NaN
1    1000.00
2        NaN
3     750.01
4        NaN
5        NaN
Name: Sales, dtype: object

Hmm. That was not what I expected. For some reason, the string values were cleaned up but the other values were turned into NaN . That’s a big problem.

To be honest, this is exactly what happened to me and I spent way more time than I should have trying to figure out what was going wrong. I eventually figured it out and will walk through the issue here so you can learn from my struggles!

The twitter thread from Ted Petrou and comment from Matt Harrison summarized my issue and identified some useful pandas snippets that I will describe below.

Basically,i assumed that an object column contains all strings.But object column can contain a mixture of all dtypes

In [18]:
df = df_orig.copy()

In [19]:
df['Sales'].apply(type)

0      <class 'int'>
1      <class 'str'>
2    <class 'float'>
3      <class 'str'>
4      <class 'int'>
5      <class 'int'>
Name: Sales, dtype: object

Ahh, this nicely shows the issue.the apply(type) code runs the type function on each value of the column.As we can see,some of them are string some are int,float .overall dtype is an object.

# first we can add a formatted column to know the type of the value

In [22]:
df['sales_type'] = df['Sales'].apply(lambda x: type(x).__name__)

In [23]:
df

,Customer,Sales,sales_type
0,Jones Brothers,500,int
1,Beta Corp,"$1,000.00",str
2,Globex Corp,300.1,float
3,Acme,$750.01,str
4,Initech,300,int
5,Hooli,250,int


In [24]:
#or,here is another compact way to check the type of data in each column
df['Sales'].apply(type).value_counts()

<class 'int'>      3
<class 'str'>      2
<class 'float'>    1
Name: Sales, dtype: int64

# Fixing the problem

When pandas tries to do a similar approach by using the str accessor, it returns an NaN instead of an error. That’s why the numeric values get converted to NaN .

The solution is to check if the value is a string, then try to clean it up. Otherwise, avoid calling string functions on a number.

The first approach is to write a custom function and use apply .

In [31]:
def clean_currency(x):
    """ If the value is a string, then remove currency symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    if isinstance(x,str):
        return(x.replace(',','').replace('$',''))
    return(x)

This function will check if the supplied value is a string and if it is, will remove all the characters we don’t need. If it is not a string, then it will return the original value.

Here is how we call it and convert the results to a float. I also show the column with the types:

In [36]:
df['Sales'] = df['Sales'].apply(clean_currency).astype('float')
df['sales_type'] = df['Sales'].apply(lambda x: type(x).__name__)

In [37]:
df

,Customer,Sales,sales_type,sales_Type
0,Jones Brothers,500.00,float,float
1,Beta Corp,1000.00,float,float
2,Globex Corp,300.10,float,float
3,Acme,750.01,float,float
4,Initech,300.00,float,float
5,Hooli,250.00,float,float


# Before finishing up, I’ll show a final example of how this can be accomplished using a lambda function:

In [38]:
df = df_orig.copy()

In [39]:
df

,Customer,Sales
0,Jones Brothers,500
1,Beta Corp,"$1,000.00"
2,Globex Corp,300.1
3,Acme,$750.01
4,Initech,300
5,Hooli,250


In [40]:
df['Sales'] = df['Sales'].apply(lambda x: x.replace(',','').replace('$','')
                               if isinstance(x,str) else x).astype(float)

In [41]:


df

,Customer,Sales
0,Jones Brothers,500.00
1,Beta Corp,1000.00
2,Globex Corp,300.10
3,Acme,750.01
4,Initech,300.00
5,Hooli,250.00


# Summary


# The pandas object data type is commonly used to store strings. However, you can not assume that the data types in a column of pandas objects will all be strings. This can be especially confusing when loading messy currency data that might include numeric values with symbols as well as integers and floats.

# It is quite possible that naive cleaning approaches will inadvertently convert numeric values to NaN . This article shows how to use a couple of pandas tricks to identify the individual types in an object column, clean them and convert them to the appropriate numeric value.